# 2. Spectrum Estimation, Crab Nebula

We will be picking up from where we left in the previous tutorial.   
In this one, we will be estimating the spectrum (i.e. the flux as a function of the energy) of the Crab Nebula.

In [ ]:
# - basic imports (numpy, astropy, regions, matplotlib)
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from regions import PointSkyRegion, CircleSkyRegion
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import logging
import warnings

# - Gammapy's imports
from gammapy.maps import Map, MapAxis, WcsGeom, RegionGeom
from gammapy.data import DataStore, Observation
from gammapy.datasets import SpectrumDataset, Datasets
from gammapy.makers import (
    SpectrumDatasetMaker,
    WobbleRegionsFinder,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.modeling.models import (
    PowerLawSpectralModel,
    LogParabolaSpectralModel,
    SkyModel,
    create_crab_spectral_model,
)
from gammapy.modeling import Fit
from gammapy.estimators import FluxPointsEstimator
from gammapy.stats import WStatCountsStatistic

# - setting up logging and ignoring warnings
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
warnings.filterwarnings("ignore")

## 2.1. Spectrum Extraction

Let us start from the previous homework, but let us do it with `Gammapy`!    
We aim at building histograms of the events in the _on_ and _off_ regions as a function of the energy. By estimating the _spectrum_, we mean we would like to find a description, through an analytical function, of this distribution in energy.

In [ ]:
# let us load the observations again
datastore = DataStore.from_dir("../acme_magic_odas_data/data/CrabNebula")
observations = datastore.get_observations(required_irf=["rad_max", "aeff", "edisp"])
print(len(observations))

### 2.1.1. The `RAD_MAX` table

In general, with increasing energy, the spatial resolution of the telescope will improve. This means that we might use tighter $\theta^2$ cuts (i.e. smaller _on_ and _off_ regions), the idea being that the $\theta^2$ histogram will become much more peaked as the energy increases (could you observe this in Exercise 1.2?). 

Pre-optimised $\theta^2$ cuts are available in the `rad_max` attribute of each observation. They correspond to the `RAD_MAX` data unit in the `.fits` file. We plot the optimised $\theta^2$ cut in each bin, and, for comparison, the $\theta^2$ cut we used in the previous tutorial.

In [ ]:
fig, ax = plt.subplots()
rad_max = observations[0].rad_max
rad_max.plot_rad_max_vs_energy(
    ax=ax,
    ls="--",
    marker=".",
    label="optimised " + r"$\theta^2$" + " cut",
    drawstyle="steps-mid",
)
ax.axhline(0.2, ls="--", lw=1, label="fixed cut (previous tutorial)", color="crimson")
ax.legend()
plt.show()

### 2.1.2. From `Observations` to `Datasets`

We will now delegate all the steps of spectrum extraction to `Gammapy` (we will basically let `Gammapy` do the homework we had in the previous tutorial).

In [ ]:
# let us define the parameters of the spectrum extraction
# - we need to specify only the center of the on region,
# its radius will be fetched from the RAD_MAX table.
crab_coords = SkyCoord.from_name("Crab Nebula")
on_region = PointSkyRegion(crab_coords)

# - let us define the energy axes over which we want:
# -- to bin the counts (estimated energies) and
energy_axis = MapAxis.from_energy_bounds(
    10, 1e5, nbin=20, per_decade=False, unit="GeV", name="energy"
)
# -- to interpolate the IRFs (true energies)
energy_axis_true = MapAxis.from_energy_bounds(
    10, 1e5, nbin=28, per_decade=False, unit="GeV", name="energy_true"
)

# let us create an empty dataset with this spatial and energy structure / binning
geom = RegionGeom.create(region=on_region, axes=[energy_axis])
dataset_empty = SpectrumDataset.create(geom=geom, energy_axis_true=energy_axis_true)

In [ ]:
dataset_maker = SpectrumDatasetMaker(
    containment_correction=False, selection=["counts", "exposure", "edisp"]
)
# use 3 off regions to estimate the background
region_finder = WobbleRegionsFinder(n_off_regions=3)
bkg_maker = ReflectedRegionsBackgroundMaker(region_finder=region_finder)

datasets = Datasets()

for observation in observations:
    dataset = dataset_maker.run(
        dataset_empty.copy(name=str(observation.obs_id)), observation
    )
    dataset_on_off = bkg_maker.run(dataset, observation)
    datasets.append(dataset_on_off)

We turned our observations in `DataSets`. What do they contain?

In [ ]:
datasets[1].peek()

## 2.2 Spectrum Fit

We see the array of _on_ and _off_ counts, and the two IRF we previously examined.    
How should we use them, and what does it mean to estimate a spectrum?


To obtain a measurement of the specturm, it is commonly assumed that a simple analytical function describes the gamma-ray (differential) flux $\Phi$ vs energy. The most common example is a power-law

$$
    \frac{{\rm d}\phi}{{\rm d}E}(E; \Phi_0, \Gamma, E_0)\,[{\rm TeV}^{-1}\,{\rm cm}^{-2}\,{\rm s}^{-1}] = 
    \Phi_0 \left(\frac{E}{E_0}\right)^{-\Gamma}
    \tag{2.1}
$$

where $\Phi_0$ is the flux normalization at the reference energy $E_0$, $\Gamma$ is the spectral index (slope of the power law), and $E_0$ is the reference energy (commonly chosen near the middle of the fitted energy range).  

Another commonly used spectral model is the log-parabola, which allows for curvature in the spectrum:  

$$
\frac{{\rm d}\phi}{{\rm d}E}(E; \Phi_0, \Gamma, \beta, E_0) =
\Phi_0 \left(\frac{E}{E_0}\right)^{-\Gamma - \beta \, \log\!\left(\frac{E}{E_0}\right)}
\tag{2.2}
$$

where $\beta$ is the curvature parameter (with $\beta = 0$ the model reduces to a pure power law).

Let us look at measurements of the spectrum of the Crab from publications.

In [ ]:
# read image
img = mpimg.imread("img/crab_sed_magic.png")
# display image
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(img)
ax.axis("off")
plt.show()

Reference: [MAGIC Collaboration (2015), JHEAP, 5, p. 30-38.](https://ui.adsabs.harvard.edu/abs/2015JHEAp...5...30A/abstract)

In [ ]:
# read image
img = mpimg.imread("img/crab_sed_hess.png")
# display image
fig, ax = plt.subplots(figsize=(10, 6))
ax.imshow(img)
ax.axis("off")
plt.show()

Reference: [H.E.S.S. Collaboration (2006), A&A, 457, 3, pp.899-915](https://ui.adsabs.harvard.edu/abs/2006A%26A...457..899A/abstract)

Let us now plot some of the examples of analytical spectra available in `Gammapy`.    

In [ ]:
energy_bounds = [80 * u.GeV, 10 * u.TeV]
pwl = PowerLawSpectralModel()
pwl.plot(energy_bounds, label="power law")
lp = LogParabolaSpectralModel(
    amplitude=7e-11 * u.Unit("TeV-1 cm-2 s-1"), reference=100 * u.GeV
)
lp.plot(energy_bounds, label="log parabola")
plt.legend()
plt.show()

We would like to find the flux vs energy (the parameters, given an assumed specrum) that would produce in our detector the same counts we observed. The issue is that flux is an absolute physical quantity, whereas the data (observed counts) are not absolute as they depend on our detector.

How can we adjust the parameters $\hat{\theta}$ of the analytical spectrum to the counts that we have extracted? That's where the IRF come into place. The IRF allow us to go from gamma-ray flux to instrument counts and vice versa. By folding (i.e. convolving) the analytical flux model with the response of the system we can obtain counts vs energy. 

The number of model (or _predicted_) counts in a given energy bin $\Delta E'$ is:

$$
g_{i}(\hat{\theta}) = 
    t_{\rm eff} \int_{\Delta E'} {\rm d}E' \int_{0}^{\infty} {\rm d}E \;
    A_{\rm eff}(E) M(E'|E) \frac{{\rm d}\phi}{{\rm d}E}(E; \hat{\theta}),
\tag{2.2}
$$

where $A_{\rm eff}(E)$ represents the effective area and $M(E'|E)$ the energy dispersion.

How can we decide which (predicted) counts best describe our model? We do this by using a likelihood, specifically the one we already introduced in the previous notebook, in Eq. 1.3. We have to take into account that we have counts as a function of the energy, therefore, we have to produce a likelihood term for each of the estimated energy bins. The total likelihood for our dataset, accounting for the observed and estimated counts in each energy bin, reads:

$$ 
\mathcal{L}(\hat{\theta}) = 
    \displaystyle \Pi_{i=1}^{N} {\rm P}(g_i(\hat{\theta}) + \alpha b_i; N_{{\rm on}, i}) {\rm P}(b_i; N_{{\rm off}, i})
\tag{2.3}
$$

where $i$ is an index that runs over $N$ estimated energy bins, and with the notation $g_i(\hat{\theta})$ denoting that the number of expected signal counts depend on the model parameters. $b_i$, the number of background counts, is typically treated as a nuisance parameter. The likelihood is maximised by varying the spectral parameters $\hat{\theta}$, and hence the number of predicted counts. `Gammapy` will take care of all these computations for us.

It is customary to fit the spectrum in an energy range that is determined by the observing conditions (especially the zenith angle that impacts the energy threshold of the instrument).

In [ ]:
# declare the energy range to be used for the fitting
e_min = 0.08 * u.TeV
e_max = 20 * u.TeV

for dataset in datasets:
    dataset.mask_fit = dataset.counts.geom.energy_mask(e_min, e_max)

# now define the model to be fitted
spectral_model = LogParabolaSpectralModel(
    amplitude=5e-12 * u.Unit("TeV-1 cm-2 s-1"),
    reference=1 * u.TeV,
    alpha=2.3 * u.Unit(""),
    beta=0.1 * u.Unit(""),
)
# trick to facilitate ULs computation
spectral_model.amplitude.min = 1e-20
spectral_model.amplitude.max = 1e-5
print(spectral_model)

# we have to assign the spectral model to a model of the sky
# that in this case will consist of a single source
model = SkyModel(spectral_model=spectral_model, name="CrabNebula")

datasets.models = [model]

We can see the parameters, and now that we have assigned the model, we can see something else.

In [ ]:
datasets[0].plot_excess()
plt.show()

We can also check the contribution of this data set to the total likelihood.

In [ ]:
datasets[0].stat_sum()

As we can see, the counts predicted by the model we specified for the data set are one order of magnitude below the observed source counts. 

To get a feeling of how the likelihood minimisation for the fit works, let us go back to the definition of the spectral analytical function and change the amplitude parameter to `5e-11 * u.Unit("TeV-1 cm-2 s-1")`. What happens to the predicted counts? And what happens to the likelihood statistics?

In [ ]:
spectral_model.amplitude.value

In [ ]:
spectral_model.amplitude.value = 5e-11

datasets[0].plot_excess()
plt.show()

In [ ]:
datasets[0].stat_sum()

You have now an intuition of how the fitting routine works. The parameters will be changed until the best agreement between observed and predicted counts is found. This agreement is statistically quantified by the likelihood, which is what is actually maximised.

In [ ]:
# run the fit!
fit = Fit()
results = fit.run(datasets=datasets)
print(results)
print(spectral_model)

In [ ]:
ax_spectrum, ax_residuals = datasets[0].plot_fit()
plt.show()

We can now plot the spectrum obtained by our fitting procedure. It is common in high-energy astrophysics, to represent instead of the differential flux the Spectral Energy Distribution (SED) obtained as $E^2\frac{{\rm d}\phi}{{\rm d}E} [{\rm TeV}^{-1}\,{\rm cm}^{-2}\,{\rm s}^{-1}]$ that represent the how the emitted power is distributed over the electromagnetic spectrum. We compare the results we have obtained with a theoretical model of the SED and with another measurement performed by MAGIC.


In [ ]:
fig, ax = plt.subplots()

plot_kwargs = {
    "sed_type": "e2dnde",
    "yunits": u.Unit("TeV cm-2 s-1"),
    "xunits": u.GeV,
}

energy_range = [80 * u.GeV, 20 * u.TeV]

spectral_model.plot(
    energy_range,
    ax=ax,
    color="crimson",
    ls="-",
    label="LST",
    **plot_kwargs,
)
spectral_model.plot_error(
    energy_range, ax=ax, facecolor="crimson", alpha=0.4, **plot_kwargs
)

crab_meyer = create_crab_spectral_model("meyer")
crab_meyer.plot(
    energy_range,
    ax=ax,
    label="Meyer et al. (2010)",
    color="k",
    ls="--",
    **plot_kwargs,
)

crab_magic = create_crab_spectral_model("magic_lp")
crab_magic.plot(
    energy_range,
    ax=ax,
    label="MAGIC Collaboration (2015)",
    color="dodgerblue",
    ls="--",
    **plot_kwargs,
)

ax.legend()

ax.set_xlabel(r"$E\,/\,{\rm GeV}$")
ax.set_ylabel(
    r"$E^2 {\rm d}\phi/{\rm d}E\,/\,({\rm TeV}\,{\rm cm}^{-2}\,{\rm s}^{-1})$"
)
ax.set_ylim([8e-13, 2e-10])
plt.show()

## 2.4. Flux Points

Finally we also compute spectral data points or _flux points_. They are basically a flux measurement over different energy bins. Broadly speaking, they have two objectives:
1) when we fit the broad-band spectrum via the likelihood maximisation - as we have just shown - we are assuming that the entire spectrum is described by a single analytic smooth function. This type of analysis is therefore not sensitive to any sharp feature in the spectrum;
2) we are adopting an analytical function to provide a broad-band flux measurements, yet, at a later stage, other astrophysicists might want to fit a physical model to our gamma-ray data. In order to do so they should have access to the data sets (counts + IRF) that we are using. In case this data are proprietary we can provide a simplified and easier way to handle flux measurements.

To compute flux points, `Gammapy` starts from the best-fit spectrum obtained from the broad-band likelihood fit and re-adjusts it spearately to the events within each energy bin. Note theat only the amplitude ($\phi_0$ in Eq. 2.1) of the spectrum is re-fitted. Basically, the spectrum we obtained from the broad-band likelihood fit is scaled up and down to adjust independently to the counts in each energy bin.

Let us use the same binning we have used in estimated energies and select an interval compatible with what was used in the likelihood fit.

In [ ]:
energy_edges = datasets[0].counts.geom.axes["energy"].edges
fpe_lst = FluxPointsEstimator(
    energy_edges=energy_edges, source="CrabNebula", selection_optional="all"
)
flux_points = fpe_lst.run(datasets=datasets)

In [ ]:
fig, ax = plt.subplots()
flux_points.plot(
    ax=ax,
    sed_type="e2dnde",
    color="darkorange",
    label="flux points",
)
flux_points.plot_ts_profiles(ax=ax, sed_type="e2dnde")
spectral_model.plot(
    energy_range,
    ax=ax,
    color="crimson",
    ls="--",
    lw=2,
    sed_type="e2dnde",
    label="broad-band likelihood fit",
)
ax.set_xlim([80, 2e4])
ax.set_xlabel(r"$E\,/\,{\rm GeV}$")
ax.set_ylabel(
    r"$E^2 {\rm d}\phi/{\rm d}E\,/\,({\rm TeV}\,{\rm cm}^{-2}\,{\rm s}^{-1})$"
)
ax.legend()
plt.show()

## Exercise 2.1.

When estimating the spectral parameters using all the observations, we have a likelihood term like in Eq. 2.3 for each of the runs.  
When we _stack_ the observations, we create a single `SpectrumDataset` from all the observations together.   
_on_ and _off_ counts are simply summed in each estimated energy bin, IRF components are averaged, weighted by the observation time.

Stack all the Crab Nebula observations (check the `Gammapy` docs) and re-perform the fit using the single dataset thus obtained. Check the results against those of the fit using all the observations simultaneously.